In [159]:
import pandas as pd
import numpy as np

In [160]:
churnData = pd.read_csv('Customer-Churn.csv')

churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [161]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [162]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'],  errors='coerce')
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7032 non-null   float64
 15  Churn             7043 non-null   object 
dtypes: float64(2), int64(2), object(12)
memory

In [163]:
churnData.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [164]:
churnData = churnData.dropna(subset=['TotalCharges'])
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [165]:
features = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
target = churnData['Churn']

from sklearn.preprocessing import Normalizer

normalizer = Normalizer()

features_normalized = pd.DataFrame(normalizer.fit_transform(features), columns=features.columns)

churnData_normalized = pd.concat([features_normalized, target.reset_index(drop=True)], axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)

accuracy


0.7611940298507462

In [166]:
churnData_normalized

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,0.023682,0.000000,0.706908,0.706908,No
1,0.017983,0.000000,0.030122,0.999384,No
2,0.016552,0.000000,0.445662,0.895048,Yes
3,0.024433,0.000000,0.022967,0.999438,No
4,0.011952,0.000000,0.422512,0.906279,Yes
...,...,...,...,...,...
7027,0.012045,0.000000,0.042561,0.999021,No
7028,0.009777,0.000000,0.014014,0.999854,No
7029,0.031620,0.000000,0.085085,0.995872,No
7030,0.012677,0.003169,0.235798,0.971714,Yes


In [167]:
for col in features.columns:
    print(f"Unique values for {col}: {features[col].unique()}")


Unique values for tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
Unique values for SeniorCitizen: [0 1]
Unique values for MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
Unique values for TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]


In [168]:
target.value_counts()

Churn
No     5163
Yes    1869
Name: count, dtype: int64

In [169]:
from sklearn.utils import resample

majority_class = churnData_normalized[churnData_normalized['Churn'] == 'No']
minority_class = churnData_normalized[churnData_normalized['Churn'] == 'Yes']

upper_minority_class = resample(minority_class,
                                        replace=True,                     
                                        n_samples=len(majority_class),  
                                        random_state=42)                  

df_upsampled = pd.concat([majority_class, upper_minority_class])


df_upsampled['Churn'].value_counts()


Churn
No     5163
Yes    5163
Name: count, dtype: int64

In [170]:
class_majority_downsampled = resample(majority_class,
                                            replace=False,    
                                            n_samples=len(minority_class), 
                                            random_state=42)

df_downsampled = pd.concat([class_majority_downsampled, minority_class])


df_downsampled['Churn'].value_counts()


Churn
No     1869
Yes    1869
Name: count, dtype: int64

In [171]:
df_upsampled

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,0.023682,0.0,0.706908,0.706908,No
1,0.017983,0.0,0.030122,0.999384,No
3,0.024433,0.0,0.022967,0.999438,No
6,0.011273,0.0,0.045656,0.998894,No
7,0.032946,0.0,0.098014,0.994640,No
...,...,...,...,...,...
2318,0.009881,0.0,0.033595,0.999387,Yes
4491,0.013148,0.0,0.085790,0.996226,Yes
3378,0.011986,0.0,0.220990,0.975202,Yes
1323,0.009896,0.0,0.016688,0.999812,Yes


In [172]:
df_downsampled

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
2343,0.028796,0.000000,0.015861,0.999459,No
2261,0.009485,0.000182,0.019052,0.999773,No
5753,0.009423,0.000314,0.033200,0.999404,No
1604,0.008913,0.000000,0.019496,0.999770,No
4242,0.015074,0.000000,0.099336,0.994940,No
...,...,...,...,...,...
7010,0.016430,0.000000,0.081878,0.996507,Yes
7015,0.022175,0.000000,0.108903,0.993805,Yes
7021,0.009334,0.009334,0.707045,0.707045,Yes
7023,0.009728,0.000000,0.014948,0.999841,Yes


In [173]:
from sklearn.metrics import accuracy_score

def train_and_evaluate(df, target_column='Churn'):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Previsione e valutazione
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Accuracy: {accuracy}")

print("Evaluation on df_upsampled:")
train_and_evaluate(df_upsampled)

print("\nEvaluation on df_downsampled:")
train_and_evaluate(df_downsampled)

Evaluation on df_upsampled:
Accuracy: 0.6819941916747337

Evaluation on df_downsampled:
Accuracy: 0.6564171122994652
